In [1]:
import torch
import torchvision.models as models

In [7]:
# Загрузка предобученной модели EfficientNet
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

models.efficientnet_b0(weights="DEFAULT")
model = models.efficientnet_b0(pretrained = True)

In [8]:
# Входные данные для модели
input_data = torch.randn(1, 3, 224, 224)

In [9]:
# Запуск модели
output = model(input_data)
print(output)

tensor([[-2.8781e-01, -2.7925e-01, -1.7340e-01, -3.6915e-01, -6.0044e-01,
         -2.8803e-01,  7.5019e-02,  3.2130e-01,  6.7799e-01, -5.2979e-01,
         -1.9018e-01,  1.6963e-01, -1.3146e-01, -5.2481e-01, -1.2365e-01,
         -3.6920e-01,  2.2017e-01, -2.7737e-01, -5.4481e-01,  1.0688e-01,
         -5.4946e-01, -3.1964e-01, -3.2250e-01, -1.4627e-01, -1.0611e-01,
         -4.4420e-01, -8.9153e-01, -1.4142e-01, -5.5071e-01, -3.6736e-01,
         -3.5693e-01, -2.4067e-01, -1.7152e-01, -3.7515e-01, -1.0901e-01,
          1.9679e-01,  5.8504e-01, -4.3635e-03, -2.7199e-01,  1.7172e-01,
         -2.7509e-01,  3.6906e-01, -3.0862e-02,  3.4152e-01, -1.2197e-01,
         -1.2535e-01, -3.0401e-01, -3.3528e-01, -6.5903e-01, -2.7790e-01,
          2.9042e-01,  2.7117e-01, -1.9847e-02, -3.0949e-01, -4.5636e-01,
          1.8157e-01,  2.0086e-01,  2.0469e-01, -1.2616e-01,  8.1466e-02,
         -4.6660e-03,  1.4216e-01,  1.7473e-01, -1.2384e-01,  5.4878e-02,
          1.6398e-02,  4.6359e-02, -1.

In [10]:
!pip install onnx onnxruntime
import onnx
import onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.9 MB/s eta 0:00:00


In [11]:
torch.onnx.export(model, input_data, "efficientnet_b0.onnx")

In [12]:
# Загрузка ONNX-модели
onnx_model = onnx.load("efficientnet_b0.onnx")

In [13]:
# Создание исполнителя ONNX-модели
ort_session = onnxruntime.InferenceSession("efficientnet_b0.onnx")

In [14]:
# Запуск модели из файла ONNX
ort_inputs = {ort_session.get_inputs()[0].name: input_data.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print("Результат запуска модели EfficientNet_b0 в ONNX:", ort_outs)

Результат запуска модели EfficientNet_b0 в ONNX: [array([[-4.56834614e-01, -2.18119308e-01,  1.46136180e-01,
        -6.93139806e-02,  4.00468379e-01,  8.11320096e-02,
         8.15163553e-02,  5.37495911e-01,  4.27408129e-01,
         4.80015159e-01,  5.30015111e-01,  2.69227952e-01,
         1.51983768e-01,  6.20617449e-01,  1.28902584e-01,
         5.73904932e-01,  3.59073997e-01,  7.49841154e-01,
         8.70042682e-01,  4.11618441e-01,  8.33990574e-01,
         1.30424511e+00,  1.43400264e+00,  6.28603041e-01,
         5.18168032e-01,  2.29709595e-01,  2.54558325e-02,
         2.34262317e-01,  3.70094925e-01, -3.21426272e-01,
        -2.54591465e-01,  1.60022467e-01, -3.16076756e-01,
         2.65176177e-01,  3.38889122e-01, -1.86752304e-01,
        -1.33619472e-01, -3.24218005e-01,  1.22665823e-01,
         5.37933886e-01,  2.87722409e-01, -2.14548558e-02,
         3.65316749e-01, -1.30534753e-01,  1.94240510e-01,
        -1.13430724e-01,  4.52660799e-01,  1.74857497e-01,
      

In [15]:
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("efficientnet_b0.onnx")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 1.8 MB/s eta 0:00:00


In [16]:
output = torch_model(input_data)
print(output)

tensor([[-4.5683e-01, -2.1812e-01,  1.4614e-01, -6.9315e-02,  4.0047e-01,
          8.1131e-02,  8.1516e-02,  5.3750e-01,  4.2741e-01,  4.8002e-01,
          5.3001e-01,  2.6923e-01,  1.5198e-01,  6.2062e-01,  1.2890e-01,
          5.7391e-01,  3.5907e-01,  7.4984e-01,  8.7004e-01,  4.1162e-01,
          8.3399e-01,  1.3042e+00,  1.4340e+00,  6.2860e-01,  5.1817e-01,
          2.2971e-01,  2.5456e-02,  2.3426e-01,  3.7010e-01, -3.2143e-01,
         -2.5459e-01,  1.6002e-01, -3.1608e-01,  2.6518e-01,  3.3889e-01,
         -1.8675e-01, -1.3362e-01, -3.2422e-01,  1.2267e-01,  5.3793e-01,
          2.8772e-01, -2.1454e-02,  3.6532e-01, -1.3053e-01,  1.9424e-01,
         -1.1343e-01,  4.5266e-01,  1.7486e-01, -7.1192e-02, -9.0029e-02,
          3.3532e-01, -5.6432e-01,  3.2253e-01,  2.6263e-01,  6.3058e-01,
          3.5242e-01,  4.0547e-01,  1.7326e-01,  1.1445e-01,  8.2794e-01,
         -5.2393e-02,  2.4772e-01, -2.8246e-01,  3.7878e-01,  3.3214e-01,
          1.8793e-01,  6.6181e-02,  5.